# 实例缓存

在文件系统实现类中，如果标记为可缓存的（属性 `.cachable`），则其实例将存储在类属性中，以快速查找而不是需要重新生成可能代价昂贵的连接和会话。缓存中的键是创建实例的参数的令牌化表示。缓存本身（属性 `._cache`）目前是一个简单的字典，但未来可能是 LRU 或更复杂的东西，以便微调实例的生命周期。

由于文件可能会保留写缓存和读缓冲区，实例缓存在某些情况下可能会导致过多的内存使用；但通常，文件的 `close` 方法将被调用，丢弃数据。只有当还存在未完成的事务或捕获的回溯时，才可能预见这会成为问题。

要禁用实例缓存，即获取一个不在缓存中的新实例，即使对于可缓存的类，也可以传递 `skip_instance_cache=True`。

## 列表缓存

对于某些实现，获取文件列表（即 ls 和任何依赖于它的操作）是昂贵的。这些实现使用类似于字典的 {class}`fsspec.dircache.DirCache` 实例来管理列表。

缓存允许通过 `listings_expiry_time` 参数进行基于时间的条目过期，或者通过 `max_paths` 参数进行 LRU 过期。这些可以设置在任何使用列表缓存的实现实例上；或者要完全跳过缓存，使用 `use_listings_cache=False`。当目标位置已知为易失性的，因为它正在从其他来源写入时，这将是合适的。

当 `fsspec` 实例向后端写入时，会调用 `invalidate_cache` 方法，以便后续列出给定路径将强制刷新。此外，一些方法如 `ls` 有一个 `refresh` 参数，以强制再次获取列表。